In [17]:
import pika
import uuid
import random
import json

In [5]:
rabbitmq_host = 'localhost'
rabbitmq_port = 5672  # Default RabbitMQ port
rabbitmq_username = 'jackpotAdmin'
rabbitmq_password = 'jackpotPassword'
rabbitmq_queue = 'reward-queue'

In [6]:
credentials = pika.PlainCredentials(username=rabbitmq_username, password=rabbitmq_password)
connection_params = pika.ConnectionParameters(host=rabbitmq_host, port=rabbitmq_port, credentials=credentials)

In [10]:
experiment_id = "1-EG"
arms = ["picture_1", "picture_2"]

In [11]:
# Send arm pulled and reward obtained messages

In [43]:
arm_selected = random.choice(arms)
sess_id = uuid.uuid4()

message_body = {
    "reward": {
        "arm_selected": arm_selected,
        "experiment_id": experiment_id,
        "session_id": str(sess_id),
        "timestamp": "2023-12-08T12:34:56.789Z",
        "context": {
            "1": "0",
            "2": "1",
            "3": "0", 
            "4": "1", 
            "5": "1", 
            "6": "0", 
            "7": "1", 
            "8": "1"
        }
    },
    "operation": "create"
}

connection = pika.BlockingConnection(connection_params)
channel = connection.channel()
channel.basic_publish(exchange='', routing_key=rabbitmq_queue, body=json.dumps(message_body))


collected_reward_message_body = {
    "reward": {
        "experiment_id": experiment_id,
        "session_id": str(sess_id),
        "timestamp": "2023-12-08T12:34:56.789Z"
    },
    "operation": "update"
}

connection = pika.BlockingConnection(connection_params)
channel = connection.channel()
channel.basic_publish(exchange='', routing_key=rabbitmq_queue, body=json.dumps(collected_reward_message_body))

In [ ]:
connection.close()